In [11]:
from gurobipy import *
import pandas as pd
import numpy as np
import math

## Improved Model with Stamina Recovery Rate

In [2]:
df = pd.read_csv("GSW_player_update.csv")
df = df[df['selected']==1]
print(df)

    selected            Player Pos  Fantasy_point  Time_limit      s      v
0          1    Andre Iguodala   G           3.67       30.54  0.120  0.060
1          1    Andrew Wiggins   F           2.48       48.00  0.052  0.047
2          1     Chris Chiozza   G           2.04       29.01  0.070  0.065
3          1        Damion Lee   G           2.01       35.70  0.056  0.036
4          1    Draymond Green   F           4.14       44.82  0.092  0.062
5          1    Gary Payton II   G           2.52       23.65  0.107  0.087
7          1  Jonathan Kuminga   C          -0.24       29.60  0.030  0.030
8          1      Jordan Poole   G           2.38       34.96  0.068  0.068
10         1      Kevon Looney   C           1.19       25.14  0.047  0.047
12         1   Nemanja Bjelica   C           4.26       29.30  0.145  0.105
13         1   Otto Porter Jr.   F           3.40       30.98  0.110  0.095
14         1     Stephen Curry   G           4.30       47.45  0.091  0.051


In [3]:
# parameters set-up
df = df[df["selected"] == 1]

F = df["Fantasy_point"].to_list()
# print("F:",F,"\n")
# F = [6.4,3.2,1.4,0.6,0,0.1,2.1,0.2,-0.4,0.3,2.1,-1.1]


# separate data into 3 groups based on the position
# z_gp1 = []
# z_gp2 = []
# z_gp3 = []
# p_gp1 = []
# p_gp2 = []
# p_gp3 = []
no_gp1 = []
no_gp2 = []
no_gp3 = []
F_gp1 = []
F_gp2 = []
F_gp3 = []
s_gp1 = []
s_gp2 = []
s_gp3 = []
v_gp1 = []
v_gp2 = []
v_gp3 = []
for i in range(len(df)):
    if df.iloc[i,2] == 'G':
#         z_gp1.append([1,0,0])
#         p_gp1.append([df.iloc[i,4]*1.5,0,0])
        no_gp1.append(i+1)
        F_gp1.append(F[i])
        s_gp1.append(df.iloc[i,5])
        v_gp1.append(df.iloc[i,6])
    elif df.iloc[i,2] == 'F':
#         z_gp2.append([0,1,0])
#         p_gp2.append([0,df.iloc[i,4]*1.5,0])     
        no_gp2.append(i+1) 
        F_gp2.append(F[i])
        s_gp2.append(df.iloc[i,5])
        v_gp2.append(df.iloc[i,6])
    else:
#         z_gp3.append([0,0,1])
#         p_gp3.append([0,0,df.iloc[i,4]*1.5])
        no_gp3.append(i+1)
        F_gp3.append(F[i])
        s_gp3.append(df.iloc[i,5])
        v_gp3.append(df.iloc[i,6])

# print("z:",z_gp1,"\t",z_gp2,"\t",z_gp3,"\n")
# print("p:",p_gp1,"\t",p_gp2,"\t",p_gp3,"\n")
print("no:",no_gp1,"\t",no_gp2,"\t",no_gp3,"\n")
print("F:",F_gp1,"\t",F_gp2,"\t",F_gp3,"\n")
print("s:",s_gp1,"\t",s_gp2,"\t",s_gp3,"\n")
print("v:",v_gp1,"\t",v_gp2,"\t",v_gp3,"\n")

no: [1, 3, 4, 6, 8, 12] 	 [2, 5, 11] 	 [7, 9, 10] 

F: [3.67, 2.04, 2.01, 2.52, 2.38, 4.3] 	 [2.48, 4.14, 3.4] 	 [-0.24, 1.19, 4.26] 

s: [0.12, 0.07, 0.055999999999999994, 0.107, 0.068, 0.091] 	 [0.052000000000000005, 0.092, 0.11] 	 [0.03, 0.047, 0.145] 

v: [0.06, 0.065, 0.036000000000000004, 0.087, 0.068, 0.051] 	 [0.047, 0.062, 0.095] 	 [0.03, 0.047, 0.105] 



In [4]:
# take center as an example
# improved 2
# z = z_gp3
# p = p_gp3
no = no_gp3
F = F_gp3
s = s_gp3
I = 48
v = v_gp3

# the best value when each player (state) has been chosen to play at each minute (stage)
V = np.zeros((I,len(F)))
# record the selecting process in each minute (stage)
path  = []
# the corresponding fantacy points at each minute (stage)
f = []
for i in range(I):
    fi = []
    for j in range(len(F)):
        fj = []
        for k in range(len(F)):
            fj.append(0)
        fi.append(fj)
    f.append(fi)
# print(f)

# DP
for i in range(I): # at the i-th minute
    i_path = []
    for j in range(len(F)): # if the j-th player is selected
        
        if i == 0: 
            for k in range(len(F)): # the FP of the k-th player
                if k == j: # the FP of the selected player
                    f[i][j][k] = F[j]-s[j]
                else: # the FP of the unselected players
                    f[i][j][k] = F[k]
            pre_select = 0        
            V[i][j] = f[i][j][j] # the best FP of selecting player j
            
        else:
            # compare which of the previous selection (h) maximize the value in this stage
            v_comp = np.zeros(len(F)).tolist()
            for h in range(len(F)): 
                v_comp[h] = V[i-1][h] + f[i-1][h][j]-s[j]
            V[i][j] = max(v_comp)
            pre_select = v_comp.index(V[i][j])
                
            for k in range(len(F)): # the FP of the k-th player
                if k == j: # the FP of the selected player
                    f[i][j][k] = f[i-1][pre_select][j]-s[j]
                else: # the FP of the unselected players
                    f[i][j][k] = min(f[i-1][pre_select][k]+v[k],F[k])
                    
        i_path.append(pre_select)
        
    path.append(i_path)

# print("V",V)
print("path",path)
# print("f",f)

# the best sum of FP in this method
tmp = V[-1].tolist()
opt_V = max(tmp)
print("Optimal value:",opt_V)

# the corresponding solution
last_select = tmp.index(opt_V)
# print(last_select)
solution = np.zeros(48).tolist()
for i in range(len(path)):
    k = len(path)-1-i
#     print(k)
    solution[k] = last_select
    last_select = path[k][last_select]
#     print(last_select)
print("Solution:",solution)
for i in range(len(solution)):
    print("Minute",i+1,":  No.",no[solution[i]],"\t")

path [[0, 0, 0], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 1], [2, 2, 2], [2, 2, 1], [2, 2, 2], [2, 2, 1], [2, 2, 2], [2, 2, 1], [2, 2, 2], [2, 2, 1], [2, 2, 2], [2, 2, 1], [2, 2, 2], [1, 2, 1], [2, 2, 2], [1, 1, 1], [2, 2, 1], [2, 2, 2], [2, 2, 1], [2, 2, 2], [2, 2, 1], [2, 2, 2], [1, 2, 1], [2, 2, 2], [1, 2, 1], [2, 2, 2], [1, 1, 1], [2, 2, 1], [2, 2, 2]]
Optimal value: 87.86900000000006
Solution: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2]
Minute 1 :  No. 10 	
Minute 2 :  No. 10 	
Minute 3 :  No. 10 	
Minute 4 :  No. 10 	
Minute 5 :  No. 10 	
Minute 6 :  No. 10 	
Minute 7 :  No. 10 	
Minute 8 :  No. 10 	
Minute 9 :  No. 10 	
Minute 10 :  No. 10 	
Minute 11 :  No. 10 	
Minute 12 :  No. 10 	
Minute 13 :  No. 10 	
M

In [5]:
# take guard as an example
# improved 2
no = no_gp1
F = F_gp1
s = s_gp1
v = v_gp1
I = 48

# print("no",no)
# print("F",F)
# print("s",s)

j_pair = []
no_pair = []
F_pair = []
s_pair = []
v_pair = []
for j1 in range(len(F)):
    for j2 in range(j1+1,len(F)):
        j_pair.append([j1,j2])
        no_pair.append([no[j1],no[j2]])
        F_pair.append([F[j1],F[j2]])
        s_pair.append([s[j1],s[j2]])
        v_pair.append([v[j1],v[j2]])


# print("\nj_pair",j_pair)
# print("no_pair",no_pair)
# print("F_pair",F_pair)
# print("s_pair",s_pair)
# print("v_pair",v_pair)

# the best value when each player (state) has been chosen to play at each minute (stage)
V = np.zeros((I,len(F_pair)))
# record the selecting process in each minute (stage)
path  = []
# the corresponding fantacy points at each minute (stage)
f = []
for i in range(I):
    fi = []
    for j in range(len(F_pair)):
        fj = []
        for k in range(len(F)):
            fj.append(0)
        fi.append(fj)
    f.append(fi)
# print(f)

# DP
for i in range(I): # at the i-th minute
    i_path = []
    for j in range(len(F_pair)): # if the j-th pair of players are selected
        select = j_pair[j]
        if i == 0: 
            for k in range(len(F)):  # the FP of the k-th player
                if k == select[0]:  # the FP of the 1st selected player
                    f[i][j][k] = F[select[0]]-s[select[0]]
                elif k == select[1]:   # the FP of the 2nd selected player                 
                    f[i][j][k] = F[select[1]]-s[select[1]]
                else: # the FP of the unselected players
                    f[i][j][k] = F[k]
            pre_select = 0        
            V[i][j] = f[i][j][select[0]] + f[i][j][select[1]] # the best FP of selecting player j
            
        else:
            # compare which of the previous selection (h) maximize the value in this stage
            v_comp = np.zeros(len(F_pair)).tolist()
            for h in range(len(F_pair)): 
                v_comp[h] = V[i-1][h] + f[i-1][h][select[0]]-s[select[0]] + f[i-1][h][select[1]]-s[select[1]]
            V[i][j] = max(v_comp)
            pre_select = v_comp.index(V[i][j])
                
            for k in range(len(F)): # the FP of the k-th player
                if k == select[0]:  # the FP of the 1st selected player
                    f[i][j][k] = f[i-1][pre_select][select[0]]-s[select[0]]
                elif k == select[1]:   # the FP of the 2nd selected player                 
                    f[i][j][k] = f[i-1][pre_select][select[1]]-s[select[1]]
                else: # the FP of the unselected players
                    f[i][j][k] = min(f[i-1][pre_select][k]+v[k],F[k])
                    
        i_path.append(pre_select)
        
    path.append(i_path)

# print("V",V)
# print("path",path)
# print("f",f)

# the best sum of FP in this method
tmp = V[-1].tolist()
opt_V = max(tmp)
print("Optimal value:",opt_V)

# the corresponding solution
last_select = tmp.index(opt_V)
# print(last_select)
solution = np.zeros(48).tolist()
for i in range(len(path)):
    k = len(path)-1-i
#     print(k)
    solution[k] = last_select
    last_select = path[k][last_select]
#     print(last_select)
print("Solution:",solution)
for i in range(len(solution)):
    print("Minute",i+1,":  No.",no_pair[solution[i]][0],"\tNo.",no_pair[solution[i]][1])

Optimal value: 245.04999999999953
Solution: [4, 4, 4, 4, 4, 4, 4, 4, 13, 4, 13, 4, 13, 4, 13, 4, 4, 14, 13, 14, 2, 14, 13, 3, 13, 3, 13, 3, 13, 3, 13, 3, 13, 3, 12, 14, 4, 12, 4, 12, 12, 8, 2, 12, 7, 4, 12, 4]
Minute 1 :  No. 1 	No. 12
Minute 2 :  No. 1 	No. 12
Minute 3 :  No. 1 	No. 12
Minute 4 :  No. 1 	No. 12
Minute 5 :  No. 1 	No. 12
Minute 6 :  No. 1 	No. 12
Minute 7 :  No. 1 	No. 12
Minute 8 :  No. 1 	No. 12
Minute 9 :  No. 6 	No. 12
Minute 10 :  No. 1 	No. 12
Minute 11 :  No. 6 	No. 12
Minute 12 :  No. 1 	No. 12
Minute 13 :  No. 6 	No. 12
Minute 14 :  No. 1 	No. 12
Minute 15 :  No. 6 	No. 12
Minute 16 :  No. 1 	No. 12
Minute 17 :  No. 1 	No. 12
Minute 18 :  No. 8 	No. 12
Minute 19 :  No. 6 	No. 12
Minute 20 :  No. 8 	No. 12
Minute 21 :  No. 1 	No. 6
Minute 22 :  No. 8 	No. 12
Minute 23 :  No. 6 	No. 12
Minute 24 :  No. 1 	No. 8
Minute 25 :  No. 6 	No. 12
Minute 26 :  No. 1 	No. 8
Minute 27 :  No. 6 	No. 12
Minute 28 :  No. 1 	No. 8
Minute 29 :  No. 6 	No. 12
Minute 30 :  No. 1 	

In [6]:
# take forward as an example
# improved 2
no = no_gp2
F = F_gp2
s = s_gp2
v = v_gp2
I = 48
print("no",no)
# print("F",F)
# print("s",s)


j_pair = []
no_pair = []
F_pair = []
s_pair = []
v_pair = []
for j1 in range(len(F)):
    for j2 in range(j1+1,len(F)):
        j_pair.append([j1,j2])
        no_pair.append([no[j1],no[j2]])
        F_pair.append([F[j1],F[j2]])
        s_pair.append([s[j1],s[j2]])
        v_pair.append([v[j1],v[j2]])


# print("\nj_pair",j_pair)
# print("no_pair",no_pair)
# print("F_pair",F_pair)
# print("s_pair",s_pair)
# print("v_pair",v_pair)

# the best value when each player (state) has been chosen to play at each minute (stage)
V = np.zeros((I,len(F_pair)))
# record the selecting process in each minute (stage)
path  = []
# the corresponding fantacy points at each minute (stage)
f = []
for i in range(I):
    fi = []
    for j in range(len(F_pair)):
        fj = []
        for k in range(len(F)):
            fj.append(0)
        fi.append(fj)
    f.append(fi)
# print(f)

# DP
for i in range(I): # at the i-th minute
    i_path = []
    for j in range(len(F_pair)): # if the j-th pair of players are selected
        select = j_pair[j]
        if i == 0: 
            for k in range(len(F)):  # the FP of the k-th player
                if k == select[0]:  # the FP of the 1st selected player
                    f[i][j][k] = F[select[0]]-s[select[0]]
                elif k == select[1]:   # the FP of the 2nd selected player                 
                    f[i][j][k] = F[select[1]]-s[select[1]]
                else: # the FP of the unselected players
                    f[i][j][k] = F[k]
            pre_select = 0        
            V[i][j] = f[i][j][select[0]] + f[i][j][select[1]] # the best FP of selecting player j
            
        else:
            # compare which of the previous selection (h) maximize the value in this stage
            v_comp = np.zeros(len(F_pair)).tolist()
            for h in range(len(F_pair)): 
                v_comp[h] = V[i-1][h] + f[i-1][h][select[0]]-s[select[0]] + f[i-1][h][select[1]]-s[select[1]]
            V[i][j] = max(v_comp)
            pre_select = v_comp.index(V[i][j])
                
            for k in range(len(F)): # the FP of the k-th player
                if k == select[0]:  # the FP of the 1st selected player
                    f[i][j][k] = f[i-1][pre_select][select[0]]-s[select[0]]
                elif k == select[1]:   # the FP of the 2nd selected player                 
                    f[i][j][k] = f[i-1][pre_select][select[1]]-s[select[1]]
                else: # the FP of the unselected players
                    f[i][j][k] = min(f[i-1][pre_select][k]+v[k],F[k])
                    
        i_path.append(pre_select)
        
    path.append(i_path)

# print("V",V)
# print("path",path)
# print("f",f)

# the best sum of FP in this method
tmp = V[-1].tolist()
opt_V = max(tmp)
print("Optimal value:",opt_V)

# the corresponding solution
last_select = tmp.index(opt_V)
# print(last_select)
solution = np.zeros(48).tolist()
for i in range(len(path)):
    k = len(path)-1-i
#     print(k)
    solution[k] = last_select
    last_select = path[k][last_select]
#     print(last_select)
print("Solution:",solution)
for i in range(len(solution)):
    print("Minute",i+1,":  No.",no_pair[solution[i]][0],"\tNo.",no_pair[solution[i]][1])

no [2, 5, 11]
Optimal value: 228.8949999999998
Solution: [2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 1, 0, 2, 0, 1, 1, 0, 2, 0, 1, 1, 0, 2, 1, 0, 1, 0, 2, 1, 0, 1, 0, 2, 1, 0, 1, 0, 2, 1, 2]
Minute 1 :  No. 5 	No. 11
Minute 2 :  No. 5 	No. 11
Minute 3 :  No. 5 	No. 11
Minute 4 :  No. 5 	No. 11
Minute 5 :  No. 5 	No. 11
Minute 6 :  No. 5 	No. 11
Minute 7 :  No. 2 	No. 5
Minute 8 :  No. 5 	No. 11
Minute 9 :  No. 2 	No. 5
Minute 10 :  No. 5 	No. 11
Minute 11 :  No. 2 	No. 5
Minute 12 :  No. 5 	No. 11
Minute 13 :  No. 2 	No. 5
Minute 14 :  No. 5 	No. 11
Minute 15 :  No. 2 	No. 5
Minute 16 :  No. 5 	No. 11
Minute 17 :  No. 2 	No. 5
Minute 18 :  No. 5 	No. 11
Minute 19 :  No. 2 	No. 11
Minute 20 :  No. 2 	No. 5
Minute 21 :  No. 5 	No. 11
Minute 22 :  No. 2 	No. 5
Minute 23 :  No. 2 	No. 11
Minute 24 :  No. 2 	No. 11
Minute 25 :  No. 2 	No. 5
Minute 26 :  No. 5 	No. 11
Minute 27 :  No. 2 	No. 5
Minute 28 :  No. 2 	No. 11
Minute 29 :  No. 2 	No. 11
Minute 30 :  No. 2 	No. 5
Minute 3

## Compared with DP Algorithm

In [15]:
# take center as an example
# improved 2
# z = z_gp3
# p = p_gp3
no = no_gp3
F = F_gp3
I = 12
s = [ math.ceil(i/3) for i in s_gp3]
v = [ math.ceil(i/3) for i in v_gp3]

# the best value when each player (state) has been chosen to play at each minute (stage)
V = np.zeros((I,len(F)))
# record the selecting process in each minute (stage)
path  = []
# the corresponding fantacy points at each minute (stage)
f = []
for i in range(I):
    fi = []
    for j in range(len(F)):
        fj = []
        for k in range(len(F)):
            fj.append(0)
        fi.append(fj)
    f.append(fi)
# print(f)

# DP
for i in range(I): # at the i-th minute
    i_path = []
    for j in range(len(F)): # if the j-th player is selected
        
        if i == 0: 
            for k in range(len(F)): # the FP of the k-th player
                if k == j: # the FP of the selected player
                    f[i][j][k] = F[j]-s[j]
                else: # the FP of the unselected players
                    f[i][j][k] = F[k]
            pre_select = 0        
            V[i][j] = f[i][j][j] # the best FP of selecting player j
            
        else:
            # compare which of the previous selection (h) maximize the value in this stage
            v_comp = np.zeros(len(F)).tolist()
            for h in range(len(F)): 
                v_comp[h] = V[i-1][h] + f[i-1][h][j]-s[j]
            V[i][j] = max(v_comp)
            pre_select = v_comp.index(V[i][j])
                
            for k in range(len(F)): # the FP of the k-th player
                if k == j: # the FP of the selected player
                    f[i][j][k] = f[i-1][pre_select][j]-s[j]
                else: # the FP of the unselected players
                    f[i][j][k] = min(f[i-1][pre_select][k]+v[k],F[k])
                    
        i_path.append(pre_select)
        
    path.append(i_path)

# print("V",V)
print("path",path)
# print("f",f)

# the best sum of FP in this method
tmp = V[-1].tolist()
opt_V = max(tmp)
print("Optimal value:",opt_V)

# the corresponding solution
last_select = tmp.index(opt_V)
# print(last_select)
solution = np.zeros(I).tolist()
for i in range(len(path)):
    k = len(path)-1-i
#     print(k)
    solution[k] = last_select
    last_select = path[k][last_select]
#     print(last_select)
print("Solution:",solution)
for i in range(len(solution)):
    print("Minute",i+1,":  No.",no[solution[i]],"\t")

path [[0, 0, 0], [2, 2, 2], [2, 2, 2], [2, 2, 1], [2, 2, 2], [2, 2, 1], [2, 2, 2], [2, 2, 1], [2, 2, 2], [2, 2, 1], [2, 2, 2], [2, 2, 1]]
Optimal value: 17.769999999999996
Solution: [2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]
Minute 1 :  No. 10 	
Minute 2 :  No. 10 	
Minute 3 :  No. 9 	
Minute 4 :  No. 10 	
Minute 5 :  No. 9 	
Minute 6 :  No. 10 	
Minute 7 :  No. 9 	
Minute 8 :  No. 10 	
Minute 9 :  No. 9 	
Minute 10 :  No. 10 	
Minute 11 :  No. 9 	
Minute 12 :  No. 10 	


In [16]:
# take guard as an example
# improved 2
no = no_gp1
F = F_gp1
I = 12
s = [ math.ceil(i/3) for i in s_gp1]
v = [ math.ceil(i/3) for i in v_gp1]

# print("no",no)
# print("F",F)
# print("s",s)

j_pair = []
no_pair = []
F_pair = []
s_pair = []
v_pair = []
for j1 in range(len(F)):
    for j2 in range(j1+1,len(F)):
        j_pair.append([j1,j2])
        no_pair.append([no[j1],no[j2]])
        F_pair.append([F[j1],F[j2]])
        s_pair.append([s[j1],s[j2]])
        v_pair.append([v[j1],v[j2]])


# print("\nj_pair",j_pair)
# print("no_pair",no_pair)
# print("F_pair",F_pair)
# print("s_pair",s_pair)
# print("v_pair",v_pair)

# the best value when each player (state) has been chosen to play at each minute (stage)
V = np.zeros((I,len(F_pair)))
# record the selecting process in each minute (stage)
path  = []
# the corresponding fantacy points at each minute (stage)
f = []
for i in range(I):
    fi = []
    for j in range(len(F_pair)):
        fj = []
        for k in range(len(F)):
            fj.append(0)
        fi.append(fj)
    f.append(fi)
# print(f)

# DP
for i in range(I): # at the i-th minute
    i_path = []
    for j in range(len(F_pair)): # if the j-th pair of players are selected
        select = j_pair[j]
        if i == 0: 
            for k in range(len(F)):  # the FP of the k-th player
                if k == select[0]:  # the FP of the 1st selected player
                    f[i][j][k] = F[select[0]]-s[select[0]]
                elif k == select[1]:   # the FP of the 2nd selected player                 
                    f[i][j][k] = F[select[1]]-s[select[1]]
                else: # the FP of the unselected players
                    f[i][j][k] = F[k]
            pre_select = 0        
            V[i][j] = f[i][j][select[0]] + f[i][j][select[1]] # the best FP of selecting player j
            
        else:
            # compare which of the previous selection (h) maximize the value in this stage
            v_comp = np.zeros(len(F_pair)).tolist()
            for h in range(len(F_pair)): 
                v_comp[h] = V[i-1][h] + f[i-1][h][select[0]]-s[select[0]] + f[i-1][h][select[1]]-s[select[1]]
            V[i][j] = max(v_comp)
            pre_select = v_comp.index(V[i][j])
                
            for k in range(len(F)): # the FP of the k-th player
                if k == select[0]:  # the FP of the 1st selected player
                    f[i][j][k] = f[i-1][pre_select][select[0]]-s[select[0]]
                elif k == select[1]:   # the FP of the 2nd selected player                 
                    f[i][j][k] = f[i-1][pre_select][select[1]]-s[select[1]]
                else: # the FP of the unselected players
                    f[i][j][k] = min(f[i-1][pre_select][k]+v[k],F[k])
                    
        i_path.append(pre_select)
        
    path.append(i_path)

# print("V",V)
# print("path",path)
# print("f",f)

# the best sum of FP in this method
tmp = V[-1].tolist()
opt_V = max(tmp)
print("Optimal value:",opt_V)

# the corresponding solution
last_select = tmp.index(opt_V)
# print(last_select)
solution = np.zeros(I).tolist()
for i in range(len(path)):
    k = len(path)-1-i
#     print(k)
    solution[k] = last_select
    last_select = path[k][last_select]
#     print(last_select)
print("Solution:",solution)
for i in range(len(solution)):
    print("Minute",i+1,":  No.",no_pair[solution[i]][0],"\tNo.",no_pair[solution[i]][1])

Optimal value: 54.29
Solution: [4, 12, 4, 12, 4, 12, 4, 12, 4, 12, 4, 4]
Minute 1 :  No. 1 	No. 12
Minute 2 :  No. 6 	No. 8
Minute 3 :  No. 1 	No. 12
Minute 4 :  No. 6 	No. 8
Minute 5 :  No. 1 	No. 12
Minute 6 :  No. 6 	No. 8
Minute 7 :  No. 1 	No. 12
Minute 8 :  No. 6 	No. 8
Minute 9 :  No. 1 	No. 12
Minute 10 :  No. 6 	No. 8
Minute 11 :  No. 1 	No. 12
Minute 12 :  No. 1 	No. 12


In [17]:
# take forward as an example
# improved 2
no = no_gp2
F = F_gp2
I = 12
s = [ math.ceil(i/3) for i in s_gp2]
v = [ math.ceil(i/3) for i in v_gp2]
print("no",no)
# print("F",F)
# print("s",s)


j_pair = []
no_pair = []
F_pair = []
s_pair = []
v_pair = []
for j1 in range(len(F)):
    for j2 in range(j1+1,len(F)):
        j_pair.append([j1,j2])
        no_pair.append([no[j1],no[j2]])
        F_pair.append([F[j1],F[j2]])
        s_pair.append([s[j1],s[j2]])
        v_pair.append([v[j1],v[j2]])


# print("\nj_pair",j_pair)
# print("no_pair",no_pair)
# print("F_pair",F_pair)
# print("s_pair",s_pair)
# print("v_pair",v_pair)

# the best value when each player (state) has been chosen to play at each minute (stage)
V = np.zeros((I,len(F_pair)))
# record the selecting process in each minute (stage)
path  = []
# the corresponding fantacy points at each minute (stage)
f = []
for i in range(I):
    fi = []
    for j in range(len(F_pair)):
        fj = []
        for k in range(len(F)):
            fj.append(0)
        fi.append(fj)
    f.append(fi)
# print(f)

# DP
for i in range(I): # at the i-th minute
    i_path = []
    for j in range(len(F_pair)): # if the j-th pair of players are selected
        select = j_pair[j]
        if i == 0: 
            for k in range(len(F)):  # the FP of the k-th player
                if k == select[0]:  # the FP of the 1st selected player
                    f[i][j][k] = F[select[0]]-s[select[0]]
                elif k == select[1]:   # the FP of the 2nd selected player                 
                    f[i][j][k] = F[select[1]]-s[select[1]]
                else: # the FP of the unselected players
                    f[i][j][k] = F[k]
            pre_select = 0        
            V[i][j] = f[i][j][select[0]] + f[i][j][select[1]] # the best FP of selecting player j
            
        else:
            # compare which of the previous selection (h) maximize the value in this stage
            v_comp = np.zeros(len(F_pair)).tolist()
            for h in range(len(F_pair)): 
                v_comp[h] = V[i-1][h] + f[i-1][h][select[0]]-s[select[0]] + f[i-1][h][select[1]]-s[select[1]]
            V[i][j] = max(v_comp)
            pre_select = v_comp.index(V[i][j])
                
            for k in range(len(F)): # the FP of the k-th player
                if k == select[0]:  # the FP of the 1st selected player
                    f[i][j][k] = f[i-1][pre_select][select[0]]-s[select[0]]
                elif k == select[1]:   # the FP of the 2nd selected player                 
                    f[i][j][k] = f[i-1][pre_select][select[1]]-s[select[1]]
                else: # the FP of the unselected players
                    f[i][j][k] = min(f[i-1][pre_select][k]+v[k],F[k])
                    
        i_path.append(pre_select)
        
    path.append(i_path)

# print("V",V)
# print("path",path)
# print("f",f)

# the best sum of FP in this method
tmp = V[-1].tolist()
opt_V = max(tmp)
print("Optimal value:",opt_V)

# the corresponding solution
last_select = tmp.index(opt_V)
# print(last_select)
solution = np.zeros(I).tolist()
for i in range(len(path)):
    k = len(path)-1-i
#     print(k)
    solution[k] = last_select
    last_select = path[k][last_select]
#     print(last_select)
print("Solution:",solution)
for i in range(len(solution)):
    print("Minute",i+1,":  No.",no_pair[solution[i]][0],"\tNo.",no_pair[solution[i]][1])

no [2, 5, 11]
Optimal value: 12.820000000000002
Solution: [2, 0, 2, 1, 2, 0, 1, 2, 0, 1, 0, 2]
Minute 1 :  No. 5 	No. 11
Minute 2 :  No. 2 	No. 5
Minute 3 :  No. 5 	No. 11
Minute 4 :  No. 2 	No. 11
Minute 5 :  No. 5 	No. 11
Minute 6 :  No. 2 	No. 5
Minute 7 :  No. 2 	No. 11
Minute 8 :  No. 5 	No. 11
Minute 9 :  No. 2 	No. 5
Minute 10 :  No. 2 	No. 11
Minute 11 :  No. 2 	No. 5
Minute 12 :  No. 5 	No. 11
